This notebook is used for consolidating all the CSVs generated from the "Environment - Creating Training Data" notebook. We consolidate the raw files into an SQL database because they are too large to fit into memory. We can then complete transformations on the raw data and restore that into the SQL database as a new table.

In [1]:
import glob
import os
import pandas as pd
import numpy as np
import sqlite3
import datetime

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.layers import advanced_activations
from keras import initializers

import theano

In [3]:
# Load the generated CSV files to an SQL DB

con = sqlite3.connect("SquareBrawlTrainingRawSQLite.db")

folders = glob.glob('training_data\\TrainOn\*')

#trains = pd.DataFrame()

for folder in folders:
    print(folder)
    trains = pd.DataFrame()
    folder = folder + '\\*'
    files = glob.glob(folder)
    sfiles = sorted(files, key=os.path.getctime)
    for sfile in sfiles:
        train = pd.read_csv(sfile)
        trains = pd.concat([trains, train])
    
    trains = trains.reset_index().drop('index',axis=1)
    trains.to_sql(name='SquareBrawlTrainingRaw', con=con, if_exists='append')


training_data\TrainOn\20160616_2  - close enough


c:\program files\anaconda3\envs\sb\lib\site-packages\pandas\core\generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


training_data\TrainOn\20160617
training_data\TrainOn\20160619
training_data\TrainOn\20160620
training_data\TrainOn\20160621
training_data\TrainOn\20170622
training_data\TrainOn\20170623
training_data\TrainOn\20170626 - model
training_data\TrainOn\20170627 - model


In [3]:
#

def score_delta_index(chunk, column_name):
    '''
    Grabs the row index for when in the 
    incrememnts in the players score happen.
    Allows us to circumvent the reward 
    allocation problem
    '''
    rows = chunk.shape[0]
    prev_score = 0
    prev_row = 0
    kills_index = []
    
    for row in range(rows):
        if chunk.loc[row, column_name] != prev_score:
            kills_index.append([prev_row, row])
            prev_score = chunk.loc[row, column_name]
            prev_row = row + 1
    del kills_index[0] # delete [0,0]
    
    return kills_index

In [13]:
def td_lambda_adjustments(chunk, adjustment_column, kills_index, reward, decay, td_lookback):
    '''
    Allows for the artificial adjustment of the reward 
    metric by allocating discounted value back based on
    player kills
    '''
    # TD lamnda adjustments for kills
    for kill in kills_index:
        killpoint = kill[1]
        exponent = td_lookback - 1
        if killpoint - td_lookback < 0:
            continue
        else:
            for row in range(killpoint - td_lookback, killpoint):
                chunk.loc[row, adjustment_column] = reward * decay ** exponent
                exponent -= 1
        
    return chunk

In [14]:
# connect to SQL DB and pull from raw table

con = sqlite3.connect("SquareBrawlTrainingRawSQLite.db")

SQL_STATEMENT = '''SELECT
    episode,
    p1_x,
    " p1_y",
    p2_x,
    " p2_y",
    right_of, 
    left_of, 
    above, 
    below, 
    x_clear, 
    y_clear, 
    cd1,
    cd2, 
    aim_down, 
    aim_left, 
    aim_up, 
    aim_right,
    action,
    p1_score,
    p2_score
FROM
SquareBrawlTrainingRaw
'''


In [15]:
'''
TABLES

TD25_decay9_pos_neg_reward_1_looking_reward
TD25_decay9_pos_neg_reward_1
TD5_decay9_pos_neg_reward_1_looking_reward
TD10_decay9_pos_neg_reward_1_looking_reward
short_TD4_decay06_pos_neg_reward_1_looking_reward

'''

'\nTABLES\n\nTD25_decay9_pos_neg_reward_1_looking_reward\nTD25_decay9_pos_neg_reward_1\nTD5_decay9_pos_neg_reward_1_looking_reward\nTD10_decay9_pos_neg_reward_1_looking_reward\nshort_TD4_decay06_pos_neg_reward_1_looking_reward\n'

In [16]:
def aim_reward(df, reward_value):
    '''
    Creates reward if the agent is pointing their 
    gun in the direction of the opponent 
    '''
    above = df['above'] * df['aim_up']
    below = df['below'] * df['aim_down']
    right = df['right_of'] * df['aim_right']
    left = df['left_of'] * df['aim_left']
    
    df['aim_rewards'] = (above + below)*df['y_clear'] + (right + left)*df['x_clear']
    df['aim_rewards'] = df['aim_rewards'].apply(lambda x: reward_value if x == 1 else 0)
    df['aim_rewards'] = df['aim_rewards'].shift(-1).fillna(0)
    
    return df['aim_rewards']

In [17]:
#-------TD LAMBDA REWARDS ADJUSTED---------#
td_lookback = 4
decay = 0.6
reward_kill = 1
reward_death = -1


# load database with all raw training_data
chunksizes = 500000
training_chunks_raw = pd.read_sql(SQL_STATEMENT, con, chunksize=chunksizes)

#create new database for adjusted rewards
td_lambda_storage = sqlite3.connect("SquareBrawlSQLite_AdjustedRewards.db")

total_kills = []
total_deaths = []
episode = 0

for chunk in training_chunks_raw:
    chunk['reward_kill_adjusted'] = 0
    chunk['reward_death_adjusted'] = 0
    chunk['aim_rewards'] = 0
    
    #chunk = chunk[chunk['p2_score']<50]
    chunk = chunk.reset_index().drop('index',axis=1)
    
    # rewards for looking at opponent
    chunk['aim_rewards'] = aim_reward(chunk, 0.01)
    
    # grab kills
    kills_index = score_delta_index(chunk,'p1_score')
    total_kills.extend(kills_index)
    chunk = td_lambda_adjustments(chunk, 'reward_kill_adjusted', kills_index, reward_kill, decay, td_lookback)
    
    # grab deaths
    deaths_index = score_delta_index(chunk, 'p2_score')
    total_deaths.extend(deaths_index)
    chunk = td_lambda_adjustments(chunk, 'reward_death_adjusted', deaths_index, reward_death, decay, td_lookback)
    
    chunk['adjusted_rewards_total'] = chunk['reward_kill_adjusted'] + chunk['reward_death_adjusted']

    chunk.to_sql(name='short_TD4_decay06_pos_neg_reward_1_looking_reward', con=td_lambda_storage, if_exists='append')
    
    print('Saving', episode)
    
    episode += 1
    
    

c:\program files\anaconda3\envs\sb\lib\site-packages\pandas\core\generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


Saving 0
Saving 1
Saving 2
Saving 3
Saving 4
Saving 5
Saving 6


In [18]:
print(len(total_kills), len(total_deaths))
print(len(total_kills)/len(total_deaths)*100)

226 854
26.46370023419204


In [19]:
# Training the model based the generated data


def keras_model(inputs, dropouts=0.05, opti=1e-6, clipping=None, clipval=None):
    keras_model = Sequential()
    
    keras_model.add(Dense(128, input_dim=inputs, 
                          kernel_initializer=initializers.RandomNormal(mean=1.0, stddev=0.1), 
                          activation='relu'))
    #keras_model.add(advanced_activations.LeakyReLU(alpha=0.01))
    #keras_model.add(advanced_activations.ELU(alpha=1))
    keras_model.add(Dropout(dropouts))
    
    keras_model.add(Dense(256, input_dim=inputs, 
                          kernel_initializer=initializers.RandomNormal(mean=1.0, stddev=0.1),
                          activation='relu'))
    #keras_model.add(advanced_activations.LeakyReLU(alpha=0.01))
    #keras_model.add(advanced_activations.ELU(alpha=1))
    keras_model.add(Dropout(dropouts))
    
    keras_model.add(Dense(512, input_dim=inputs, 
                          kernel_initializer=initializers.RandomNormal(mean=1.0, stddev=0.1),
                          activation='relu'))
    #keras_model.add(advanced_activations.LeakyReLU(alpha=0.01))
    #keras_model.add(advanced_activations.ELU(alpha=1))
    keras_model.add(Dropout(dropouts))
    
    keras_model.add(Dense(256, input_dim=inputs, 
                          kernel_initializer=initializers.RandomNormal(mean=1.0, stddev=0.1),
                          activation='relu'))
    #keras_model.add(advanced_activations.LeakyReLU(alpha=0.01))
    #keras_model.add(advanced_activations.ELU(alpha=1))
    keras_model.add(Dropout(dropouts))
    
    keras_model.add(Dense(128, input_dim=inputs, 
                          kernel_initializer=initializers.RandomNormal(mean=1.0, stddev=0.1),
                          activation='relu'))
    #keras_model.add(advanced_activations.LeakyReLU(alpha=0.01))
    #keras_model.add(advanced_activations.ELU(alpha=1))
    keras_model.add(Dropout(dropouts))
    
    keras_model.add(Dense(1))
    adam = Adam(lr=opti, clipnorm=clipping, clipvalue=clipval)
    keras_model.compile(loss='mse', optimizer='adam')
    return keras_model

In [20]:
#connect to DB

con = sqlite3.connect("SquareBrawlSQLite_AdjustedRewards.db")

SQL_STATEMENT = '''SELECT
    p1_x,
    " p1_y",
    p2_x,
    " p2_y",
    right_of, 
    left_of, 
    above, 
    below, 
    x_clear, 
    y_clear, 
    cd1,
    cd2, 
    aim_down, 
    aim_left, 
    aim_up, 
    aim_right,
    action,
    reward_kill_adjusted,
    aim_rewards,
    adjusted_rewards_total,
    p2_score
FROM
    TD10_decay9_pos_neg_reward_1_looking_reward
'''

In [21]:
chunk.shape

(9735, 24)

In [22]:
# remove memory errors from dataset AFTER the rewards have been allocated
def drop_memory_errors(df):
    '''
    Some features generate errors so the whole row would be blank.
    This function removes those errrors from the training set.
    '''
    p1 = df['p1_x'] + df[' p1_y']
    p2 = df['p2_x'] + df[' p2_y']
    df['memory_error'] = p1*p2
    df['memory_error'] = df['memory_error']
    df = df[df['memory_error'] != 0]
    df.drop('memory_error', axis=1, inplace=True)
    return df

In [28]:
chunksizes = 500
drop_cols = ['reward_kill_adjusted' , 'aim_rewards', 'adjusted_rewards_total', 'p2_score']

training_chunks_clean = pd.read_sql(SQL_STATEMENT, con, chunksize=chunksizes)

# grab out input shape size
for chunk in training_chunks_clean:
    input_shape = chunk.shape[1] - len(drop_cols) # minus 1 for y target which is to be dropped
    print('input shape', input_shape)
    break

input shape 17


In [29]:
Qas = keras_model(inputs=input_shape, dropouts=0.1, opti=1e-6, clipping=None, clipval=None)
#Qas.load_weights('model_weights//total_rewards_v2//1500000_model_backup_20170626_1614.keras')

save_location = 'model_weights//testing//'

In [30]:
Qas.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 128)               2304      
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               131328    
__________

In [31]:
chunksizes = 1000000
training_chunks_clean = pd.read_sql(SQL_STATEMENT, con, chunksize=chunksizes)

In [32]:
# train keras model and save weights periodically
episode = 1
total_rows = 0
adjusted_rows = 0

#grab out input shape size
for chunk in training_chunks_clean:
    
    #drop memory errors
    total_rows += chunk.shape[0]
    chunk = drop_memory_errors(chunk)
    chunk = chunk[chunk['p2_score']<50]
    adjusted_rows += chunk.shape[0]
    
    #randomize sample
    #chunk = chunk.sample(frac=1, random_state=1337) # randomly mix everything
    
    y_val = chunk['reward_kill_adjusted'] #+ chunk['aim_rewards']
    y = np.array(y_val)
    
    for drop in drop_cols:
        chunk.drop(drop, axis=1, inplace=True)

    training_set = np.array(chunk)
    Qas.fit(training_set, y, epochs=150, verbose=1)
    
    name =  save_location + str(episode*chunksizes) + '_model_backup_' + datetime.datetime.today().strftime('%Y%m%d_%H%M') + '.keras'
    Qas.save_weights(name)
    print('Saving ', name)
    
    episode += 1
    
    print('----------------------| ', episode*chunksizes/3005000*100, '% done |---------------------------------')

c:\program files\anaconda3\envs\sb\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Epoch 1/150
21540/21540 [==============================] - 6s - loss: 111137485233854152704.0000     
Epoch 2/150
21540/21540 [==============================] - 6s - loss: 42656036448617758720.0000     
Epoch 3/150
21540/21540 [==============================] - 6s - loss: 18559712370211590144.0000     
Epoch 4/150
21540/21540 [==============================] - 6s - loss: 17785592289470771200.0000     
Epoch 5/150
21540/21540 [==============================] - 6s - loss: 7178877202275830784.0000     
Epoch 6/150
21540/21540 [==============================] - 6s - loss: 3825409194559627264.0000     
Epoch 7/150
21540/21540 [==============================] - 6s - loss: 2773497644701135360.0000     
Epoch 8/150
21540/21540 [==============================] - 6s - loss: 2096969625110181632.0000     
Epoch 9/150
21540/21540 [==============================] - 6s - loss: 1630730253757091584.0000     
Epoch 10/150
21540/21540 [==============================] - 6s - loss: 1327245439448315392.0000

19654/19654 [==============================] - 61s - loss: 0.0135    - ETA: 0s - loss:
Epoch 33/150
19654/19654 [==============================] - 62s - loss: 0.0135    
Epoch 34/150
19654/19654 [==============================] - 62s - loss: 0.0135    
Epoch 35/150
19654/19654 [==============================] - 62s - loss: 0.0135    - ETA:
Epoch 36/150
19654/19654 [==============================] - 62s - loss: 0.0135    
Epoch 37/150
19654/19654 [==============================] - 62s - loss: 0.0135    
Epoch 38/150
19654/19654 [==============================] - 62s - loss: 0.0135    
Epoch 39/150
19654/19654 [==============================] - 61s - loss: 0.0135    - ETA: 1s 
Epoch 40/150
19654/19654 [==============================] - 62s - loss: 0.0135    
Epoch 41/150
19654/19654 [==============================] - 62s - loss: 0.0135    
Epoch 42/150
19654/19654 [==============================] - 62s - loss: 0.0135    
Epoch 43/150
19654/19654 [==============================] - 62s - l

19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 75/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 76/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 77/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 78/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 79/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 80/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 81/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 82/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 83/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 84/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 85/150
19415/19415 [==============================] - 101s - loss: 0.0139   
Epoch 86/150
1941

5721/5721 [==============================] - 34s - loss: 0.0124    
Epoch 122/150
5721/5721 [==============================] - 34s - loss: 0.0123    
Epoch 123/150
5721/5721 [==============================] - 34s - loss: 0.0124    
Epoch 124/150
5721/5721 [==============================] - 34s - loss: 0.0124    
Epoch 125/150
5721/5721 [==============================] - 34s - loss: 0.0124    
Epoch 126/150
5721/5721 [==============================] - 34s - loss: 0.0124    
Epoch 127/150
5721/5721 [==============================] - 34s - loss: 0.0124    
Epoch 128/150
5721/5721 [==============================] - 34s - loss: 0.0124    
Epoch 129/150
5721/5721 [==============================] - 34s - loss: 0.0124    
Epoch 130/150
5721/5721 [==============================] - 34s - loss: 0.0183    
Epoch 131/150
5721/5721 [==============================] - 34s - loss: 0.0124    
Epoch 132/150
5721/5721 [==============================] - 34s - loss: 0.0124    
Epoch 133/150
5721/5721 [=====

In [20]:
name =  save_location + str(episode*chunksizes) + '_model_backup_' + datetime.datetime.today().strftime('%Y%m%d_%H%M') + '.keras'
Qas.save_weights(name)

In [45]:
print(adjusted_rows, total_rows)
print(adjusted_rows / total_rows)

1342532 3085000
0.4351805510534846
